<a href="https://colab.research.google.com/github/DataSavvyYT/AI-engineering-course/blob/main/04_mcp/02_mcp_server_event.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -q mcp uvicorn nest_asyncio

In [14]:
import threading
import uvicorn
from mcp.server.fastmcp import FastMCP

In [15]:
# 1. Create the Server
mcp = FastMCP("MathServer")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

In [16]:
# 2. Function to run the server
def start_server():
    # runs on http://0.0.0.0:8000 by default
    mcp.run(transport="sse")

In [17]:
# 3. Start it in a background thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

print("Server is running in the background on port 8000...")

Server is running in the background on port 8000...


In [18]:
import asyncio
import nest_asyncio
from mcp.client.sse import sse_client
from mcp import ClientSession

In [19]:
# Fix Colab's async loop
nest_asyncio.apply()

In [20]:
async def run_client():
    # Connect to the local server we started above
    # Note: FastMCP at /sse by default for SSE transport
    async with sse_client("http://localhost:8000/sse") as (read, write):
        async with ClientSession(read, write) as session:

            # Initialize connection
            await session.initialize()

            # 1. List Tools
            tools = await session.list_tools()
            print(f"\nConnected! Found tools: {[t.name for t in tools.tools]}")

            # 2. Call 'add'
            result = await session.call_tool("add", arguments={"a": 10, "b": 20})
            print(f"Add Result: {result.content[0].text}")

            # 3. Call 'multiply'
            result = await session.call_tool("multiply", arguments={"a": 5, "b": 5})
            print(f"Multiply Result: {result.content[0].text}")

In [21]:
# Run it
await run_client()

INFO:     127.0.0.1:35174 - "GET /sse HTTP/1.1" 200 OK
INFO:     127.0.0.1:35182 - "POST /messages/?session_id=911b2dd083fd4d59b29c3fb625ad6be6 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:35182 - "POST /messages/?session_id=911b2dd083fd4d59b29c3fb625ad6be6 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:35182 - "POST /messages/?session_id=911b2dd083fd4d59b29c3fb625ad6be6 HTTP/1.1" 202 Accepted

Connected! Found tools: ['add', 'multiply']
INFO:     127.0.0.1:35182 - "POST /messages/?session_id=911b2dd083fd4d59b29c3fb625ad6be6 HTTP/1.1" 202 Accepted
Add Result: 30
INFO:     127.0.0.1:35182 - "POST /messages/?session_id=911b2dd083fd4d59b29c3fb625ad6be6 HTTP/1.1" 202 Accepted
Multiply Result: 25
